# SDE-GAN Training on GPU

Train a Neural SDE-GAN for synthetic multi-asset price path generation.

**Runtime**: Go to Runtime > Change runtime type > **T4 GPU** (free tier) or A100 (Colab Pro).

**Configs** (auto-detected by GPU):
- **T4/L4**: Kidger-scale model (hidden=16, depth=1) + WGAN-GP + large batch.
- **A100+**: Larger model (hidden=64, depth=2) + WGAN-GP.

**Data**: You need minute-resolution parquet files (`ETH_USD.parquet`, etc.).
Three options (pick one in the data cell below):
1. **Google Drive** (recommended): upload parquets to Drive once, reuse across sessions
2. **Direct upload**: use Colab file upload widget (~623 MB, a few minutes)
3. **Download from Binance**: uses `update_historic_data` pipeline (~15-30 min)

In [ ]:
# Check GPU
import subprocess
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)
if 'A100' in result.stdout or 'V100' in result.stdout or 'H100' in result.stdout:
    GPU_TIER = 'A100'
    print('High-end GPU detected — using large model + WGAN-GP config')
else:
    GPU_TIER = 'T4'  # T4, L4, or unknown
    print('T4/L4 detected — using Kidger-scale config + WGAN-GP')

In [ ]:
# Install deps
!pip install -q jax[cuda12] equinox diffrax optax
!pip install -q dask pandas pyarrow

In [ ]:
# Clone repo
import os
if not os.path.exists('/content/quantammsim'):
    !git clone -b synthetic-price-gen https://github.com/QuantAMMProtocol/quantammsim.git /content/quantammsim
os.chdir('/content/quantammsim')
!pip install -q -e .

In [ ]:
# Verify JAX sees GPU
import jax
print(f'JAX devices: {jax.devices()}')
assert any(d.platform == 'gpu' for d in jax.devices()), 'No GPU! Change runtime type.'

In [ ]:
import jax.numpy as jnp
import numpy as np

from quantammsim.synthetic.sde_gan import (
    train_sde_gan, generate_paths, compute_daily_log_prices,
)
from quantammsim.utils.data_processing.historic_data_utils import get_historic_parquet_data

In [ ]:
# =============================================================
# DATA: pick ONE method, comment out the others
# =============================================================
tokens = ['ETH', 'BTC', 'USDC', 'PAXG']
DATA_DIR = 'quantammsim/data'
os.makedirs(DATA_DIR, exist_ok=True)

# --- Option 1: Google Drive (recommended) --------------------
# Upload parquets to Drive once (e.g. My Drive/quantammsim_data/),
# then every session just mounts and copies.
from google.colab import drive
drive.mount('/content/drive')
DRIVE_DATA = '/content/drive/My Drive/quantammsim_data'  # adjust path
for t in tokens:
    src = f'{DRIVE_DATA}/{t}_USD.parquet'
    dst = f'{DATA_DIR}/{t}_USD.parquet'
    if not os.path.exists(dst):
        !cp "{src}" "{dst}"
        print(f'Copied {t}_USD.parquet from Drive')
    else:
        print(f'{t}_USD.parquet already present')

# --- Option 2: Direct upload ---------------------------------
# Uncomment below, comment out Option 1 above.
#
# from google.colab import files
# print('Upload: ETH_USD.parquet, BTC_USD.parquet, USDC_USD.parquet, PAXG_USD.parquet')
# uploaded = files.upload()
# for name, data in uploaded.items():
#     with open(f'{DATA_DIR}/{name}', 'wb') as f:
#         f.write(data)
#     print(f'Saved {name} ({len(data)/1e6:.1f} MB)')

# --- Option 3: Download from Binance Vision ------------------
# Slow (~15-30 min) but fully automated. Uncomment below,
# comment out Options 1 & 2, and add to the pip install cell:
#   !pip install -q binance-historical-data Historic-Crypto bidask
#
# from quantammsim.utils.data_processing.historic_data_utils import update_historic_data
# data_dir_str = DATA_DIR + '/'
# for token in tokens:
#     if os.path.exists(f'{DATA_DIR}/{token}_USD.parquet'):
#         print(f'{token}_USD.parquet exists, skipping')
#         continue
#     print(f'Downloading {token}...')
#     update_historic_data(token, data_dir_str)
#     combined = f'{DATA_DIR}/combined_data/{token}_USD.parquet'
#     if os.path.exists(combined):
#         os.rename(combined, f'{DATA_DIR}/{token}_USD.parquet')
#     print(f'{token} done')

# Verify
print('\nParquet files:')
for t in tokens:
    p = f'{DATA_DIR}/{t}_USD.parquet'
    if os.path.exists(p):
        sz = os.path.getsize(p) / 1e6
        print(f'  {t}_USD.parquet ({sz:.1f} MB)')
    else:
        print(f'  {t}_USD.parquet MISSING!')

In [ ]:
# Load data
tokens = ['ETH', 'BTC', 'USDC', 'PAXG']
data_root = 'quantammsim/data'
price_df = get_historic_parquet_data(tokens, cols=['close'], root=data_root)
close_cols = [f'close_{t}' for t in tokens]
minute_prices = price_df[close_cols].values.astype(np.float64)
valid_mask = ~np.any(np.isnan(minute_prices), axis=1)
first_valid = np.argmax(valid_mask)
last_valid = len(valid_mask) - np.argmax(valid_mask[::-1])
minute_prices = minute_prices[first_valid:last_valid]
n_assets = len(tokens)
minute_prices_jnp = jnp.array(minute_prices)
daily_log = compute_daily_log_prices(minute_prices_jnp)
n_days = daily_log.shape[0]

real_returns = jnp.diff(daily_log, axis=0)
real_drift = jnp.mean(real_returns, axis=0)
real_vol = jnp.std(real_returns, axis=0)
real_corr = jnp.corrcoef(real_returns.T)

print(f'Data: {n_days} days, {n_assets} assets')
for i, t in enumerate(tokens):
    print(f'  {t}: drift={float(real_drift[i]):.6f}/day, vol={float(real_vol[i]):.6f}/day')
print(f'\nCorrelations:')
for i in range(n_assets):
    for j in range(i+1, n_assets):
        print(f'  {tokens[i]}-{tokens[j]}: {float(real_corr[i,j]):.3f}')

## Config

Auto-detected by GPU. T4 gets Kidger-scale (weight clipping), L4/A100+ gets larger model (WGAN-GP).

In [ ]:
if GPU_TIER == 'A100':
    # A100 / V100 / H100 — larger model stabilized with WGAN-GP
    CONFIG = dict(
        hidden_size=64, width_size=64, depth=2,
        noise_size=8, initial_noise_size=8,
        batch_size=1024, window_len=50,
        n_steps=20000,
        generator_lr=2e-5, discriminator_lr=1e-4,
        drift_lambda=1.0,
        gp_lambda=10.0,  # WGAN-GP instead of weight clipping
        use_reversible_heun=True,
    )
else:
    # T4 / L4 — Kidger-scale model + WGAN-GP
    CONFIG = dict(
        hidden_size=16, width_size=16, depth=1,
        noise_size=3, initial_noise_size=5,
        batch_size=1024, window_len=50,
        n_steps=10000,
        generator_lr=2e-5, discriminator_lr=1e-4,
        drift_lambda=1.0,
        gp_lambda=10.0,  # WGAN-GP instead of weight clipping
        use_reversible_heun=True,
    )

print(f'Config ({GPU_TIER}):')
for k, v in CONFIG.items():
    print(f'  {k}: {v}')

## Train

In [ ]:
import time
import equinox as eqx

SAVE_DIR = '/content/drive/My Drive/quantammsim_data'

def checkpoint(gen, vs, step):
    """Save model to Drive during training (survives disconnects)."""
    eqx.tree_serialise_leaves(f'{SAVE_DIR}/generator.eqx', gen)
    np.save(f'{SAVE_DIR}/vol_scale.npy', np.array(vs))
    print(f'  [checkpoint] saved at step {step}')

key = jax.random.PRNGKey(42)
t0 = time.time()

generator, vol_scale, history = train_sde_gan(
    minute_prices_jnp, n_assets=n_assets, key=key,
    verbose=True, checkpoint_fn=checkpoint, checkpoint_every=2000,
    **CONFIG,
)

elapsed = time.time() - t0
print(f'\nDone in {elapsed:.0f}s ({elapsed/CONFIG["n_steps"]*1000:.1f}ms/step)')

# Final save
checkpoint(generator, vol_scale, CONFIG['n_steps'])
print('Training complete, model saved to Drive.')

In [ ]:
# Load saved model (use this instead of retraining in a new session)
# Uncomment this cell and skip the Train cell above.
#
# import equinox as eqx
# from quantammsim.synthetic.sde_gan import Generator
# SAVE_DIR = '/content/drive/My Drive/quantammsim_data'
# skeleton = Generator(
#     data_size=n_assets,
#     initial_noise_size=CONFIG['initial_noise_size'],
#     noise_size=CONFIG['noise_size'],
#     hidden_size=CONFIG['hidden_size'],
#     width_size=CONFIG['width_size'],
#     depth=CONFIG['depth'],
#     use_reversible_heun=CONFIG.get('use_reversible_heun', False),
#     key=jax.random.PRNGKey(0),
# )
# generator = eqx.tree_deserialise_leaves(f'{SAVE_DIR}/generator.eqx', skeleton)
# vol_scale = jnp.array(np.load(f'{SAVE_DIR}/vol_scale.npy'))
# print(f'Loaded model from {SAVE_DIR}/')

## Evaluate

In [ ]:
y0 = daily_log[0]
key_eval = jax.random.PRNGKey(99)
N_PATHS = 2000

for horizon in [10, 30, 50, 100, 200]:
    paths = generate_paths(generator, vol_scale, y0,
                           n_days=horizon, n_paths=N_PATHS, key=key_eval)
    y0_bc = jnp.broadcast_to(y0[:, None], (n_assets, N_PATHS))[None, ...]
    full = jnp.concatenate([y0_bc, paths], axis=0)
    returns = jnp.diff(full, axis=0)
    drift = jnp.mean(returns, axis=(0, 2))
    vol = jnp.mean(jnp.std(returns, axis=2), axis=0)

    flat_ret = returns.transpose(0, 2, 1).reshape(-1, n_assets)
    gen_corr = jnp.corrcoef(flat_ret.T)

    print(f'\n--- {horizon}-day paths ---')
    for i, t in enumerate(tokens):
        rd, rv = float(real_drift[i]), float(real_vol[i])
        d, v = float(drift[i]), float(vol[i])
        dr = d / rd if abs(rd) > 1e-8 else float('inf')
        print(f'  {t}: drift={d:.6f} ({dr:.1f}x), vol={v:.6f} ({v/rv:.2f}x)')
    print(f'  Correlations (real -> gen):')
    for i in range(n_assets):
        for j in range(i+1, n_assets):
            print(f'    {tokens[i]}-{tokens[j]}: {float(real_corr[i,j]):.3f} -> {float(gen_corr[i,j]):.3f}')

## Lambda Sweep (optional)

Run this cell to sweep drift_lambda and find the optimal value for your GPU config.

In [ ]:
# Optional: sweep drift_lambda
SWEEP = False  # Set to True to run

if SWEEP:
    sweep_results = {}
    for lam in [0.0, 0.1, 0.5, 1.0, 2.0]:
        print(f'\n{"="*50}')
        print(f'drift_lambda = {lam}')
        print(f'{"="*50}')
        cfg = {**CONFIG, 'drift_lambda': lam}
        key_s = jax.random.PRNGKey(42)
        gen_s, vs_s, hist_s = train_sde_gan(
            minute_prices_jnp, n_assets=n_assets, key=key_s,
            verbose=True, **cfg,
        )
        # Quick 50d eval
        paths_s = generate_paths(gen_s, vs_s, y0, n_days=50, n_paths=1000, key=key_eval)
        y0_bc_s = jnp.broadcast_to(y0[:, None], (n_assets, 1000))[None, ...]
        full_s = jnp.concatenate([y0_bc_s, paths_s], axis=0)
        ret_s = jnp.diff(full_s, axis=0)
        d_s = jnp.mean(ret_s, axis=(0, 2))
        v_s = jnp.mean(jnp.std(ret_s, axis=2), axis=0)
        flat_s = ret_s.transpose(0, 2, 1).reshape(-1, n_assets)
        gc_s = jnp.corrcoef(flat_s.T)
        sweep_results[lam] = {
            'drift_ratios': {t: float(d_s[i])/float(real_drift[i]) if abs(float(real_drift[i])) > 1e-8 else float('inf') for i, t in enumerate(tokens)},
            'vol_ratios': {t: float(v_s[i])/float(real_vol[i]) for i, t in enumerate(tokens)},
            'eth_btc_corr': float(gc_s[0, 1]),
        }

    # Summary table
    print(f'\n{"="*60}')
    print('SWEEP SUMMARY (50-day horizon)')
    print(f'{"="*60}')
    print(f'{"lambda":>8} | {"ETH drift":>10} {"BTC drift":>10} {"ETH-BTC corr":>13}')
    print('-' * 50)
    for lam, r in sweep_results.items():
        ed = r['drift_ratios'].get('ETH', 0)
        bd = r['drift_ratios'].get('BTC', 0)
        ec = r['eth_btc_corr']
        print(f'{lam:>8.1f} | {ed:>9.1f}x {bd:>9.1f}x {ec:>12.3f} (real: 0.813)')